# Практическое задание 6: Градиентный бустинг

## Подготовка рабочей среды
Сначала установим нужные нам версии библиотек.

После установки нужных версий, **возможно,** нужно перезагрузить среду (runtime), но скорее всего вам это не понадобится

In [ ]:
!pip install gdown numpy pandas scikit-learn xgboost catboost lightgbm hyperopt matplotlib seaborn

## Используемые библиотеки

В этом задании могут понадобится четыре библиотеки (помимо sklearn), а именно:

**XGBoost**: Документация [здесь](https://xgboost.readthedocs.io/en/stable/).<br />
**LightGBM**: Документация [здесь](https://lightgbm.readthedocs.io/en/latest/index.html). Также дополнительно про установку [тут](https://pypi.org/project/lightgbm/).<br />
**Catboost**: Документация [здесь](https://catboost.ai/en/docs/). Можно найти также некоторую информацию на русском [тут](https://habr.com/ru/company/otus/blog/527554/).<br />
**HyperOpt**: Документация [здесь](http://hyperopt.github.io/hyperopt/). <br />

<font color='red'>**Внимание!**</font> Вникать и подробно читать документацию к каждой библиотеке нет необходимости! Достаточно обращаться туда для нахождения примеров.


## Напоминание, как правильно перебирать параметры


1. <font color='green'>**learning_rate**</font> $-$ **темп обучения** нашего метода. Для этого метода сетка перебора должна быть логарифмической, т.е. перебирать порядковые значения (к примеру, `[1e-3, 1e-2, 1e-1, 1]`). В большинстве случаев достаточно перебрать значения от `1e-5 до 1`.<br />

2. <font color='green'>**max_depth**</font> $-$ **максимальная глубина деревьев** в ансамбле. Вообще говоря, эта величина зависит от числа признаков, но обычно лучше растить небольшие деревья. К примеру, библиотека `CatBoost`, которую мы будем исследовать сегодня, рекомендует перебирать значения до `10` (и уточняется, что обычно оптимальная глубина лежит `от 6 до 10`).<br />

3. <font color='green'>**subsample**</font> $-$ **объем выборки**, использующийся для обучения отдельного дерева, лежит в интервале `(0, 1]`. Перебирать стоит хотя бы с шагом `0.25` <br />

4. <font color='green'>**n_estimators**</font> $-$ **количество деревьев** в ансамбле. Обычно стоит перебирать с каким-то `крупным шагом` (можно по логарифмической сетке). Здесь важно найти баланс между производительностью, временем обучения и качеством. Обычно `нескольких тысяч` деревьев бывает достаточно.<br />

<font color='red'>**NB!**</font> Учтите, что в реальных задачах необходимо следить за тем, что оптимальные значения параметров не попадают на границы интервалов, т.е. что вы нашли хотя бы локальный минимум. Если вы перебрали значения параметра от 1 до 10 и оказалось, что 10 $-$ оптимальное значение, значит следует перебрать и бОльшие числа, чтобы убедиться, что качество не улучшается дальше (или по крайней мере убедиться, что рост качества сильно замедляется и на сильное улучшение рассчитывать не стоит.


### HyperOpt

Обычнно перебор параметров и поиск по сетке это самая скучная часть работы, поскольку занимает много времени, но не гарантирует воспроизведение результата при небольшом изменении датасета, да и сетку надо переосмысливать при каждом обновлении.

Но этого можно избежать, поскольку есть библиотека, которая всё сделает за нас! Нашего спасителя зовут `HyperOpt`. На первый взгляд hyperopt делает всё то же самое, что и grid search, а именно перебирает параметры. По факту же hyperopt превращает это в задачу оптимизации, используя некоторые эвристики для ускорения сходимости процесса. К тому же, он требует лишь информацию о границе интервалов, а не сами сетки. В теории это должно помочь нам добиться лучших результатов за более короткое время. 

Примерный порядок действий:
1. Взять `любую библиотеку градиентного бустинга`.
2. Составить `сетку перебора в hyperopt`, включающую параметры n_estimators, max_depth, subsample и learning_rate в hyperopt. Вам могут понадобиться такие типы данных, как `hp.choice, hp.qloguniform, hp.uniform и hp.quniform`(можно также пользоваться np.arange). Также для округления значения типа float до целых чисел (4.0 $\to$ 4) можно использовать `scope.int`.
3. `Реализовать функцию`, которая принимает на вход словарь параметров для регрессора, и при помощи CV оценивает его качество на датасете (можно воспользоваться cross_val_score, а для ускорения поставить cv=3). 
4. Создать объект `trials=Trials()`, который будет хранить информацию о процессе оптимизации. Используя функцию fmin, можно `оптимизировать функцию`. Можно установить algo=tpe.suggest, trials=trials и max_evals, по крайней мере, 50. verbose=1 позволит видеть прогресс-бар по типу tqdm.

# Предсказание зрительских симпатий

----------------------------------------------
<font color="white" style="opacity:0.2023"></font>

В некотором царстве, некотором государстве была развита кинопромышленность. Новые фильмы в этом государстве показывают по интернету, а пользователи после просмотра могут дать фильму некоторую "награду". Наша цель $-$ предсказать число наград для фильма.

### Описание данных

В нашем распоряжении имеются следующие данные:

- **awards** $-$ количество наград, полученных фильмом от пользователей (целевое значение)  
- **potions** $-$ количество магических зелий, потраченных на создание спец-эффектов  
- **genres** $-$ жанры созданного фильма  
- **questions** $-$ количество вопросов, заданных пользователями на соответствующих форумах об этом фильме до премьеры  
- **directors** $-$ режиссеры фильма (если неизвестны, то `unknown`)  
- **filming_locations** $-$ области, в которых снимался фильм  
- **runtime** $-$ продолжительность фильма в некоторых единицах, принятых в этом государстве  
- **critics_liked** $-$ количество критиков из 100, присудивших награды фильму на предварительных закрытых показах  
- **pre-orders** $-$ количество зрителей, заранее купивших билеты на первый показ  
- **keywords** $-$ ключевые слова, описывающие содержание фильма
- **release_year** $-$ год, во котором фильм был показан (конечно, в летоисчислении этого государства)

Следующие поля появляются несколько раз с разными значениями `i`:

- **actor_i_known_movies** $-$ количество известных фильмов актера `i` (i от 0 до 2)
- **actor_i_postogramm** $-$ количество подписчиков в социальной сети "по сто грамм" актера `i` (i от 0 до 2)
- **actor_i_gender** $-$ пол актера `i` (i от 0 до 2)
- **actor_i_age** $-$ возраст актера `i` (i от 0 до 2)

### Формат данных

Данные разбиты на три части: тренировочная часть, публичные тестовые команды (которые вы можете скачать) и приватные тестовые файлы. 

Данные хранятся в виде JSONL-файлов, то есть файлов, в которых каждая строка $-$ это JSON. Каждая строка соответствует одному объекту в датасете. Ключи этого JSON соответствуют названиям переменных. В примере решения Вы можете увидеть как читать такие файлы без проблем.

**Важно!** В тестовом файле сохраняйте даннные в том же порядке, в котором они записаны в файл!

---
Вот по [этой ссылке](https://drive.google.com/file/d/1wCOyYTt-n1UU1Snf5O4ExVUuvkhkI3y_/view?usp=sharing) хранится ZIP-архив с публичными данными и шаблоном решения.

In [ ]:
# при работе в Colab'е можно использовать эту ячейку для скачивания
!gdown 1wCOyYTt-n1UU1Snf5O4ExVUuvkhkI3y_

#### Устройство архива

```
solution_template/
│
├── run.py
│   → Основной скрипт запуска решения. 
│
├── awards_prediction.py
│   → Модуль с логикой предсказаний (обучение модели, обработка данных,
│     применение ML-алгоритма и т.п.). Может содержать класс или функции для
│     подготовки данных и построения прогноза.
│
└── public_tests/
    ├── 01_boosting_movies_gt/
    │   └── target.json
    │       → Файл с эталонными значениями для публичного теста.
    │
    └── 01_boosting_movies_input/
        ├── test/
        │   └── test.jsonl
        │       → Публичный тестовый набор данных в формате JSON Lines.
        │
        └── train/
            └── train.jsonl
                → Публичный обучающий набор данных в формате JSON Lines.
```


### Решение
**<font color='red'>Внимание!</font>** Подбирать оптимальные параметры стоит **только** на локальном компьютере / в Google Colab!
В решении Вы должны использовать регрессор с оптимальными параметрами, которые вы нашли путём перебора по сетке.

В шаблонном файле `awards_prediction.py` Вы должны реализовать функцию `train_model_and_predict`, которая получает на вход папку для обучения и теста. На обучении вы обучаете ваш алгоритм, а затем возвращаете предсказания значений `awards` для всех фильмов из теста. Предсказания должны быть расположены в том же порядке, в котором они находятся в тесте (то есть, не примените где-то случайно `shuffle`).

В этом файле вы можете создавать любые дополнительные функции и методы, которые нужны вам для решения. Главное – сохранить интерфейс функции `train_model_and_predict`.

### Советы по решению
В этом задании Вы можете добиться лучшего качества при помощи:
- Предобработки датасета, выбора категориальных переменных и дополнительной фильтрации.
- Выбора лучшего метода обучения и подбора оптимальных параметров с использованием кросс-валидации.

**<font color='red'>Внимание!</font>** Учтите, что при OHE кодировании признаки на обучении и тестировании должны совпадать! Если вы примените простое `.get_dummies()` или что-то подобное, то признаки на трейне и тесте получатся разные! Так что вам, вероятно, придётся придумать способ для того, чтобы сохранить их :)  

**<font color='red'>Внимание!</font>** Нельзя исключать вероятность того, что злые силы добавили в наш датасет пропуски, поэтому лучше лишний раз как-нибудь заполнить пропущенные значения.

**Подсказка**: для работы с текстом можно воспользоваться методом TF-IDF (ключевые слова: TfIdfTransformer). Также может быть полезен CountVectorizer. Только учтите, что никто не гарантирует улучшение результата с использованием данных методов  ;)

### Разрешенные методы и библиотеки
В качестве метода обучения предлагается использовать **любой регрессор**, основанный на градиентном бустинге деревьев. Разрешается пользоваться библиотеками sklearn, xgboost, lightgbm, catboost.

**Жесткого требования использовать градиентный бустинг нет!** Градиентный бустинг является одним из лучших методов обучения на сегодняшний день, поэтому будет даже интересно, удастся ли кому-то получить макс. балл альтернативными методами.

Также разрешается пользоваться библиотекой hyperopt для подбора параметров модели (параметры стоит подбирать локально, а в систему загружать решение с подобранными параметрами).

### Оценивание

В качестве метрики качества используется значение MAE.

Баллы выставляются по следующим правилам:
- 9 баллов: $\text{MAE} ∈ [0, 2050]$,
- 7 баллов: $\text{MAE} ∈ (2050, 2100]$,
- 5 баллов: $\text{MAE} ∈ (2100, 2150]$,
- 3 баллов: $\text{MAE} ∈ (2150, 2200]$,
- 1 балла: $\text{MAE} ∈ (2200, 2300]$,
- 0 баллов: $\text{MAE} ∈ (2300, +∞]$

Значение MAE будет посчитано раздельно на публичной и приватной выборках. Количество полученных баллов на приватной выборке будет дополнительно умножено на 2. Таким образом за это задание Вы можете получить до 27 баллов (9 на публичной и 18 на приватной выборках).

### Тестирование
Скачайте ZIP-архив c шаблоном решения и разархивируйте его. Далее следуйте инструкциям по запуску тестирования.

Тесты запускаются внутри папки с шаблоном через терминал с помощью команды:
```bash
python run.py
```

Учтите, что после запуска скрипта будет создано несколько дополнительных файлов и директорий (это
связано с работой тестирующей системы).

Если всё верно, то Вы увидите что-то вроде `Mark: 1 OK, mae = [2287.341688095019]`, т.е. Вашу оценку и значение MAE на публичном датасете.

## **<font color='orange'>Задание (27 баллов)</font>**
**Данные**: датасет с наградами за фильмы

**Метрика**: MAE  

**Цели**: В данном задании следует выполнить следующие пункты:  
1. Взять `любую библиотеку`.
2. Используя предложенный датасет, `обучить регрессор` для предсказания awards (предоставляем полную свободу в настройках и выборе методов).
3. Получить качество больше порогового значения.

In [ ]:
## your efficient code here